In [1]:
# TODO

# < 1h
# Make basic version with no nlp

# Clean code, rearrange in clear steps : panda join, slipt, preprocessing, training
# clean Variable names (make sure it still works), add comments


# ~1h
# make the diy nlp with most occuring degree terms
# test if imporvements



# pandas output
#email to ask 

# test rnn

# improvements: cross check with friend list

In [2]:
import os
import urllib.request
import pandas as pd
import numpy as np

# 0. Loading the data

Downloading the data, so that we don't have to do it again if we restart the kernel

In [3]:
urls=["https://info0.s3.us-east-2.amazonaws.com/recruitment/positions.csv","https://info0.s3.us-east-2.amazonaws.com/recruitment/education.csv","https://info0.s3.us-east-2.amazonaws.com/recruitment/jobtitle_seniority.csv"]

# data folder, create if it does not exists
os.makedirs('./data/', exist_ok=True) 
    
# if .csv file does not exists, download it
for url in urls:
    path='./data/' + os.path.basename(url)
    if not os.path.exists(path):
        urllib.request.urlretrieve(url, path)
        
print('data downloaded')

data downloaded


Loading the data into pandas data frames

In [4]:
#TODO dl?
#TODO S3 bucket code?
df_positions = pd.read_csv("./data/positions.csv")
df_education = pd.read_csv("./data/education.csv")
df_seniority = pd.read_csv("./data/jobtitle_seniority.csv")

# 1. Checking the data

## 1.1 Individual data frames

In [5]:
df_positions=df_positions.sort_values(by=['user_id'])
df_positions.tail(10)

,user_id,jobtitle,startdate,enddate
162425,zzdHAVxl9iQrwom22S/FLg5+2cvffV/mNepQVJd0smgtpB...,student_senior_service_college,2012-08-01,2013-06-01
15089,zzdHAVxl9iQrwom22S/FLg5+2cvffV/mNepQVJd0smgtpB...,command_general_staff_college,2004-07-01,2005-06-01
146169,zzdHAVxl9iQrwom22S/FLg5+2cvffV/mNepQVJd0smgtpB...,engineer_company_brigade_staff_trainer,2002-04-01,2004-06-01
15923,zzdHAVxl9iQrwom22S/FLg5+2cvffV/mNepQVJd0smgtpB...,student_engineer_officer,1998-01-01,1998-10-01
20486,zzdHAVxl9iQrwom22S/FLg5+2cvffV/mNepQVJd0smgtpB...,company_commander_battalion_battalion_assistant,1998-11-01,2002-03-01
309939,zzrNxfUzwZXNkSs15haLyA4ZM3TcQvn1bQ/jHgHWG0kf/b...,partner_head_private_client_department_|_law_p...,1992-01-01,NaN
338504,zzvZxBSf81furoFl3PcSuHAG1BvSkUYANepQVJd0smgtpB...,coordinador_de_personal_embarcado,2017-12-01,NaN
334912,zzvZxBSf81furoFl3PcSuHAG1BvSkUYANepQVJd0smgtpB...,operador_|_logistics_supply_chain,2014-11-01,2015-09-01
359048,zzvZxBSf81furoFl3PcSuHAG1BvSkUYANepQVJd0smgtpB...,supervisor_de_personal_|_maritime,2016-05-01,2017-09-01
248375,zzvZxBSf81furoFl3PcSuHAG1BvSkUYANepQVJd0smgtpB...,supervisor_de_personal_|_logistics_supply_chain,2015-11-01,2016-05-01


#### Looking at the dates, linkedIn only has the year and month information, so date formatting is YYYY-MM-DD
#### Position titles and fields are seperated by '\_|\_', words are seperated by '_'

In [6]:
df_education=df_education.sort_values(by=['user_id'])
df_education.head()

,user_id,major,startdate,enddate
99508,++5SW5MI5/h8X1hMA3QnmQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,BS,1949-01-01,1953-01-01
92083,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,BS in Electronics,1973-01-01,1978-01-01
92505,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,NaN,1984-01-01,1987-01-01
133238,++6+hv3i5RAVsrWO8q5JEQ5+2cvffV/mNepQVJd0smgtpB...,Master Grande Ecole,2013-01-01,2016-01-01
5126,++6zEVtPCi83vpPTHSY2Vg5+2cvffV/mNepQVJd0smgtpB...,Bachelor of Science (B.Sc.) (ED),2001-01-01,2006-01-01


In [7]:
df_seniority=df_seniority.sort_values(by=['user_id'])
df_seniority.head()

,user_id,jobtitle,seniority
90141,++5SW5MI5/h8X1hMA3QnmQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,former_owner_presently_consultant,7.064817
71129,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,design_engineer_|_mechanical_industrial_engine...,3.331507
222292,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,owner_|_computer_network_security,7.334247
399946,++6+hv3i5RAVsrWO8q5JEQ5+2cvffV/mNepQVJd0smgtpB...,digital_communication_social_medias_activation...,4.307247
220871,++6+hv3i5RAVsrWO8q5JEQ5+2cvffV/mNepQVJd0smgtpB...,event_promoter_public_relations,1.908356


In [8]:
print(df_education.info())
print(df_positions.info())
print(df_seniority.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226184 entries, 99508 to 169422
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    226184 non-null  object
 1   major      162346 non-null  object
 2   startdate  197556 non-null  object
 3   enddate    190658 non-null  object
dtypes: object(4)
memory usage: 8.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 377585 entries, 41525 to 248375
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    377585 non-null  object
 1   jobtitle   376136 non-null  object
 2   startdate  368526 non-null  object
 3   enddate    270354 non-null  object
dtypes: object(4)
memory usage: 14.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 416295 entries, 90141 to 126315
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   

In [9]:
# give the number of DISTINCT users
def count_users(df):
    return len(pd.unique(df['user_id']))

#### As expected a person can have multiple education, and position entries

In [10]:
test_id =df_education.iat[2,0]
def get_user_history(user_id):
    print(df_education[df_education['user_id'].str.contains(user_id,regex=False,na=False)][['major', 'startdate','enddate']])
    print(df_positions[df_positions['user_id'].str.contains(user_id,regex=False,na=False)][['jobtitle', 'startdate','enddate']])
    print(df_seniority[df_seniority['user_id'].str.contains(user_id,regex=False,na=False)][['jobtitle', 'seniority']])
get_user_history(test_id)

                   major   startdate     enddate
92083  BS in Electronics  1973-01-01  1978-01-01
92505                NaN  1984-01-01  1987-01-01
                                                 jobtitle   startdate  \
9781                    owner_|_computer_network_security  1993-06-01   
106525  design_engineer_|_mechanical_industrial_engine...  1984-10-01   

           enddate  
9781           NaN  
106525  1989-05-01  
                                                 jobtitle  seniority
71129   design_engineer_|_mechanical_industrial_engine...   3.331507
222292                  owner_|_computer_network_security   7.334247


## 1.2 Missing values

#### Missing values for each df
There are more missing values for end dates because it can the current position / education program of someone

Missing values for dates are more important than for education/jobtittles to predict the age

In [11]:
#Missing values
print(df_positions.isna().sum(),'------',sep='\n')
print(df_education.isna().sum(),'------',sep='\n')
print(df_seniority.isna().sum(),'------',sep='\n')

user_id           0
jobtitle       1449
startdate      9059
enddate      107231
dtype: int64
------
user_id          0
major        63838
startdate    28628
enddate      35526
dtype: int64
------
user_id         0
jobtitle     2005
seniority       0
dtype: int64
------


#### Missing starting dates AND enddates are problematic because it makes it hard to impute data

In [12]:
df_missing_pos=df_positions[df_positions['enddate'].isna() & df_positions['startdate'].isna()]
df_missing_sen=df_seniority[df_seniority['seniority'].isna() ]
df_missing_edu=df_education[df_education['enddate'].isna() & df_education['startdate'].isna()]
print(count_users(df_missing_pos),'unique users missing job date info in df_positions')
print(count_users(df_missing_sen),'unique users missing seniority info')
print(count_users(df_missing_edu),'unique users missing education date info')



6518 unique users missing job date info in df_positions
0 unique users missing seniority info
18036 unique users missing education date info


#### This is even more true if there are no dates for both the education and job entries

In [13]:
df_missing_all=df_missing_pos.merge(df_missing_edu, how='inner',left_on=['user_id'], right_on=['user_id'])
print(count_users(df_missing_all),'unique users missing dates on all education and job info')

981 unique users missing dates on all education and job info


# 2. Splitting the data to get training data 

## dataprocessing will be done after to prevent data leakage

## 2.1 Choosing the training Data

People with highscool information have ages that are easy to predict since most people end highschool at the same age.
The same is true for Bachelors. 
As a general rule, the ealier the education is in terms of degree, the better it is to predict the age because of the smaller variability.

Because there are more people with bachelor information (1/3 of the data) we will use the starting date of the bachelor to determine someone's age for the training data

In [14]:
mask_high = df_education.major.apply(lambda x: ('highschool' in str(x).lower()) or ('high school' in str(x).lower()) )
print(count_users(df_education[mask_high]),'unique users with a highschool info')

3368 unique users with a highschool info


In [15]:
bachelors=['bachelor','bs ','ba ','b.s.','b.a.']
mask_ba = df_education.major.apply(lambda x:  any(bachelor in str(x).lower() for bachelor in bachelors))
print(count_users(df_education[mask_ba]),'unique users with a bachelor info')

37463 unique users with a bachelor info


In [16]:
bachelors=['bachelor']
mask_ba = df_education.major.apply(lambda x:  any(bachelor in str(x).lower() for bachelor in bachelors))
print(count_users(df_education[mask_ba]),'unique users with a bachelor info containing exactly \'bachelor\'')

33931 unique users with a bachelor info containing exactly 'bachelor'


#### We will use the later as it is simpler to work with, and 1/3 of the data as training data is enough.
##### In the more advanced versions, we use NLP and clustering to encode similer majors, taking into account abbreviations.

In [17]:
training_users=pd.unique(df_education[mask_ba]['user_id']).tolist()

## 2.2 Splitting the data frames

In [18]:
from sklearn.model_selection import train_test_split
def split_df(users,df):

    users_train,users_val=train_test_split( users, test_size = 0.3, random_state = 42)
    df_train=df.loc[df['user_id'].isin(users_train)]
    df_val=df.loc[df['user_id'].isin(users_val)]
    df_test=df.loc[~ df['user_id'].isin(users)]
    return df_train,df_val,df_test

df_positions_train,df_positions_val,df_positions_test=split_df(training_users,df_positions)
df_education_train,df_education_val,df_education_test=split_df(training_users,df_education)
df_seniority_train,df_seniority_val,df_seniority_test=split_df(training_users,df_seniority)


# checking that we still have every user
assert(count_users(df_positions_train)+count_users(df_positions_val)+count_users(df_positions_test)==count_users(df_positions)) #posisions does not have 100k users
assert(count_users(df_education_train)+count_users(df_education_val)+count_users(df_education_test)==count_users(df_education))
assert(count_users(df_seniority_train)+count_users(df_seniority_val)+count_users(df_seniority_test)==count_users(df_seniority))


# 3 Data processing

In [19]:
from datetime import datetime
import random
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import Pipeline
import multiprocessing as mp

In [20]:
def tokenize(text):
    return(text.split('_'))

class DataTransformer:
    # to preven data leakage
    #class constants, used for test data too, but they are only changed with the training data when fit=true
    
    avg_length_job= avg_length_edu = avg_end_job = avg_start_job= avg_end_edu = avg_start_edu =0
    majors_pipe = jobs_pipe = None
    
    def __init__(self,df_positions,df_education,df_seniority):
        self.df_positions = df_positions
        self.df_education=df_education
        self.df_seniority = df_seniority
    ### 
    def format_to_size(self,df,labels=False):
        def to_fixed_size(df,cols,size):
            for col in cols:
                df[col] = df[col].apply(lambda x: x[0:size] if (len(x)>size) else  (x + (size-len(x)) * [x[-1]]))
            
            return df
        
        return to_fixed_size(df,['enddate_y','startdate_y','seniority','enddate_x','startdate_x','major_cluster','job_cluster'],10)
    ###
    def get_labels(self,df,remove_info=False):
        df['label']=None
        def get_first_index(mylist, substring):
            for i, s in enumerate(mylist):
                if substring in s:
                      return i
            return -1
        for index, row in df.iterrows():
            i=get_first_index(row['major'],'bachelor')
            birth=row['startdate_y'][i]-18
            if i!=-1:
                df.loc[index,'label'] = birth 
                if remove_info and len(row['enddate_y'])>1 and random.random()>0.3: # remove bachelor info for training
                    df.at[index,'enddate_y'] = row['enddate_y'][:i]+row['enddate_y'][i+1:]
                    df.at[index,'startdate_y'] = row['startdate_y'][:i]+row['startdate_y'][i+1:]
                    df.at[index,'major'] =  row['major'][:i]+row['major'][i+1:]
            else:
                print('ERROR COULD NOT GET LABEL FOR USER:', row.index.name)
        return df
    ###
    def merge_pd(self,df_positions, df_education,df_seniority,labels=False,remove_info=False):
        # sorting by startdate for later:
        df_positions.sort_values(by=['startdate'],inplace=True)
        df_education.sort_values(by=['startdate'],inplace=True)
        
        df=df_positions.merge(df_seniority, how='right',left_on=['user_id','jobtitle'], right_on=['user_id','jobtitle'])
        df=df.groupby('user_id').agg(lambda x: list(x))
        df_education_temp=df_education.groupby('user_id').agg(lambda x: list(x))
        df=df.merge(df_education_temp, how='outer', left_on='user_id', right_on='user_id')
        
        df['startdate_x']=df.apply(lambda row: [max(row['enddate_y']) if pd.isnull(date) else date for date in row['startdate_x'] ] ,axis=1)
        now = datetime.now()
        today=now.year+now.month/13
        df['enddate_x']=df['enddate_x'].apply(lambda dates: [today if pd.isnull(date) else date for date in dates])
        
        if labels: # add labels
            df=self.get_labels(df,remove_info)
        return df
    ###
    def transform_pd(self,df_positions, df_education,df_seniority,fit=False):
        
        def to_lower_case(df,col):
            df.loc[:,col] =  df[col].str.lower()
            return df 
        def to_datetime(df,col):
            df.loc[:,col] =  pd.to_datetime(df[col], format='%Y-%m-%d')
            df.loc[:,col] =  df[col].dt.year+ df[col].dt.month/13
            return df
        print('changing dates and text format \n')

        df_education = df_education.pipe(to_lower_case, col='major').pipe(to_datetime,col='startdate').pipe(to_datetime,col='enddate')
        df_positions = df_positions.pipe(to_lower_case, col='jobtitle').pipe(to_datetime,col='startdate').pipe(to_datetime,col='enddate')
        df_seniority = df_seniority.pipe(to_lower_case, col='jobtitle')
        
        def fill_end_dates(df):
            now = datetime.now()
            today=now.year+now.month/13
            #print('today',today)
            before=df['enddate'].isna().sum()
            df.loc[:,'enddate']=df.loc[:,:].apply(lambda row: today if pd.isnull(row['enddate']) and (not pd.isnull(row['startdate'])) else row['enddate'],axis=1)
            print('added',before-df['enddate'].isna().sum(),'values \n')
            return df

        print('completing position end dates...')

        df_positions=fill_end_dates(df_positions)
        print('completing education end dates...')
        df_education=fill_end_dates(df_education)

        def fill_startdate(df,typ):
            df_nona=df[['startdate','enddate']].dropna(how='all',inplace=False)
            df_nona['length']=df_nona['enddate']-df_nona['startdate']
            avg_length=0
            if fit:
                if typ=='edu':
                    DataTransformer.avg_length_edu=avg_length=df_nona["length"].mean()
                else:
                    DataTransformer.avg_length_job=avg_length=df_nona["length"].mean()
            else:
                if typ=='edu':
                    avg_length=DataTransformer.avg_length_edu
                else:
                    avg_length= DataTransformer.avg_length_job
                
            print('average length: ',avg_length,'yrs')
            before=df['startdate'].isna().sum()
            df['startdate'].fillna(df['enddate']-avg_length, inplace=True)
            print('added',before-df['startdate'].isna().sum(),'values \n')
            return df
        print('completing education start dates...')
        df_education=fill_startdate(df_education,'edu')
        print('completing position end dates...')
        df_positions=fill_startdate(df_positions,'job')
        
        def fill_dates_missingall(df,typ):
            df_nona=df[['startdate','enddate']].dropna(how='all',inplace=False)
            #df_nona['length']=df_nona['enddate']-df_nona['startdate']
            
            avg_start=avg_end=0
            if fit:
                if typ=='edu':
                    DataTransformer.avg_start_edu=avg_start=df_nona["startdate"].mean()
                    DataTransformer.avg_end_edu=avg_end=df_nona["enddate"].mean()
                else:
                    DataTransformer.avg_start_job=avg_start=df_nona["startdate"].mean()
                    DataTransformer.avg_end_job=avg_end=df_nona["enddate"].mean()
            else:
                if typ=='edu':
                    avg_start=DataTransformer.avg_start_edu
                    avg_end=DataTransformer.avg_end_edu
                else:
                    avg_start=DataTransformer.avg_start_job
                    avg_end=DataTransformer.avg_end_job
            
                
            print('average start date: ',avg_start,'yrs')
            print('average end date: ',avg_end,'yrs')
            before=df['startdate'].isna().sum()
            df['startdate'].fillna(avg_start, inplace=True)
            df['enddate'].fillna(avg_end, inplace=True)
            print('added',before-df['startdate'].isna().sum(),'values \n')
            return df  
        print('completing education remaining dates...')
        df_education=fill_dates_missingall(df_education,'edu')
        print('completing position remaining dates...')
        df_positions=fill_dates_missingall(df_positions,'job')
        
        # filling missing education and positions
        df_positions['jobtitle'].fillna('positions', inplace=True)
        df_seniority['jobtitle'].fillna('positions', inplace=True)
        df_education['major'].fillna('education', inplace=True)
        
        #filling missing seniority information
        df_seniority['seniority'].fillna((df_seniority['seniority'].mean()), inplace=True)
            
        
        return df_positions,df_education,df_seniority
    
    ####
    def get_edu_clusters(self,df,fit,remove_info):
        if fit:
            processed_text=[]
            for t in df['major'].tolist():
                r=0
                if remove_info:
                    r=random.random() # if training or val data remove bachelor data to match the testing data bachelor ratio
                if (not pd.isnull(t)) and (not 'bachelor' in t.lower() or r <0.33): # removing one third of bechelors to match the rest of the data set
                    processed_text.append(t)
            # clustering pipeline   

            number_of_clusters = 10
            DataTransformer.majors_pipe = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 4))), ('majors_model', KMeans(n_clusters=number_of_clusters, init='k-means++', max_iter=50, n_init=2))])
            DataTransformer.majors_pipe.fit(processed_text)
            order_centroids = DataTransformer.majors_pipe['majors_model'].cluster_centers_.argsort()[:, ::-1]
            terms = DataTransformer.majors_pipe['vectorizer'].get_feature_names_out()
            for i in range(0,number_of_clusters):
                print("class %d:" % i),
                for ind in order_centroids[i, :10]:
                    print(' %s' % terms[ind])
                print('-------') 
                
        # add cluster info to df
        majors=df['major'].astype(str).tolist()
        major_clusters=DataTransformer.majors_pipe.predict(majors)
        df['major_cluster']=major_clusters
        
        return df
    ###
    def get_position_clusters(self,df,fit):
        if fit:
            processed_text=[]
            for t in df['jobtitle'].tolist():
           
                if (not pd.isnull(t)): # removing one third of bechelors to match the rest of the data set
                    processed_text.append(t)
            # clustering pipeline   

            number_of_clusters = 10
            DataTransformer.jobs_pipe = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 4),stop_words=['|'])), ('jobs_model', KMeans(n_clusters=number_of_clusters, init='k-means++', max_iter=50, n_init=2))])
            DataTransformer.jobs_pipe.fit(processed_text)
            order_centroids = DataTransformer.jobs_pipe['jobs_model'].cluster_centers_.argsort()[:, ::-1]
            terms = DataTransformer.jobs_pipe['vectorizer'].get_feature_names_out()
            for i in range(0,number_of_clusters):
                print("class %d:" % i),
                for ind in order_centroids[i, :10]:
                    print(' %s' % terms[ind])
                print('-------') 
                
        # add cluster info to df
        jobs=df['jobtitle'].astype(str).tolist()
        jobs_clusters=DataTransformer.jobs_pipe.predict(jobs)
        df['job_cluster']=jobs_clusters
        
        return df
    ###
    def transform(self,fit=False,labels=False,remove_info=False):
        
        self.df_education=self.get_edu_clusters(self.df_education,fit,remove_info)
        self.df_seniority=self.get_position_clusters(self.df_seniority,fit)
        
        self.df_positions,self.df_education,self.df_seniority=self.transform_pd(self.df_positions, self.df_education,self.df_seniority,fit)
        merged=self.merge_pd(self.df_positions,self.df_education,self.df_seniority,labels,remove_info)
        
        
        return self.format_to_size(merged,labels)
    
    
dataTransformer = DataTransformer(df_positions_train,df_education_train,df_seniority_train)
train_data =dataTransformer.transform(fit=True,labels=True,remove_info=True)


class 0:
 of
 bachelor
 bachelor of
 arts
 of arts
 bachelor of arts
 ba
 engineering
 bachelor of arts ba
 of arts ba
-------
class 1:
 degree
 bachelor
 bachelor degree
 in
 degree in
 bachelor degree in
 of
 degree of
 bachelor degree of
 engineering
-------
class 2:
 diploma
 masters
 degree
 in
 certificate
 bachelors
 school
 of
 high
 high school
-------
class 3:
 master
 degree
 master degree
 in
 master degree in
 degree in
 master 39 degree
 39 degree
 39
 master 39
-------
class 4:
 science
 of
 of applied
 applied science
 applied
 of applied science
 bachelor of applied science
 bachelor of applied
 bachelor
 bachelor of
-------
class 5:
 of
 business
 of business
 administration
 of business administration
 business administration
 bachelor
 bachelor of
 bachelor of business
 bachelor of business administration
-------
class 6:
 of
 master
 master of
 administration
 business
 master of business
 of business
 master of business administration
 of business administration
 

C:\Users\nguye\AppData\Local\Temp/ipykernel_13508/2790559756.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['major_cluster']=major_clusters


class 0:
 mining
 metals
 mining metals
 engineer
 manager
 engineer mining
 engineer mining metals
 intern
 senior
 intern mining
-------
class 1:
 services
 technology
 information
 information technology
 technology services
 information technology services
 manager
 engineer
 engineer information technology
 engineer information
-------
class 2:
 engineer
 education
 intern
 management
 senior
 services
 analyst
 sales
 higher
 higher education
-------
class 3:
 assistant
 education
 higher
 higher education
 manager
 research
 assistant higher education
 assistant higher
 research assistant
 administrative
-------
class 4:
 fitness
 health
 wellness
 health wellness
 wellness fitness
 health wellness fitness
 manager
 manager health
 manager health wellness
 manager health wellness fitness
-------
class 5:
 director
 marketing
 advertising
 marketing advertising
 services
 education
 executive
 sales
 management
 development
-------
class 6:
 administration
 government
 government

C:\Users\nguye\AppData\Local\Temp/ipykernel_13508/2790559756.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_cluster']=jobs_clusters
C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

changing dates and text format 

completing position end dates...
added 24017 values 

completing education end dates...
added 1815 values 

completing education start dates...
average length:  3.608725981355846 yrs
added 398 values 

completing position end dates...
average length:  3.962133030775858 yrs
added 21 values 

completing education remaining dates...
average start date:  2003.2742277481523 yrs
average end date:  2006.882953729532 yrs
added 6235 values 

completing position remaining dates...
average start date:  2010.1799402438633 yrs
average end date:  2014.1420732741742 yrs
added 1635 values 



C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [21]:
feature_cols=['enddate_y','startdate_y','seniority','enddate_x','startdate_x','major_cluster']


X=np.asarray(train_data[feature_cols].values.tolist())
# matrix dim for emsemble method
X=np.reshape(X, (len(X), X.shape[1]*X.shape[2]))

# gets rid of future warning
y=np.asarray(train_data[['label']]).reshape(-1)

# Model

#### After testing multiple models from sklearn, ensemble methods were the ones that worked the best

#### We are going to investigate Neural networks later 

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [23]:
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size = 0.3, random_state = 42)

In [24]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# defining parameter range 
# Number of trees in random forest
n_estimators = [200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x*10) for x in range(1,5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
njobs=[-1]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
              'n_jobs' : njobs
                }
   
#grid = RandomizedSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 10) 
   
# fitting the model for grid search 
#grid.fit(X_train, y_train) 
 
# print best parameter after tuning 
#print(grid.best_params_) 
#grid_predictions = grid.predict(X_val) 
   


In [25]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

In [26]:
# Scaler not necessary here but it make it easier to change model, when we have to use it
pipe = Pipeline([('scaler', StandardScaler()), ('RandomForestRegressor', RandomForestRegressor(n_estimators=200,n_jobs=-1,min_samples_split=2,max_features='auto',min_samples_leaf=2,bootstrap=True,max_depth=40))])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_val)
print('r2 score', pipe.score(X_val,y_val))
print('explained_variance_score',explained_variance_score(y_val, y_pred))
print('validation mean abs error',mean_absolute_error(y_val, y_pred))
print('validation RMSE',mean_squared_error(y_val, y_pred,squared=False))

r2 score 0.8487480640218668
explained_variance_score 0.8487680760488888
validation mean abs error 2.1513609278220915
validation RMSE 4.034577903691025


In [27]:
dataTransformer = DataTransformer(df_positions_test,df_education_test,df_seniority_test)
test_data =dataTransformer.transform(fit=False,labels=False,remove_info=False)

C:\Users\nguye\AppData\Local\Temp/ipykernel_13508/2790559756.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['major_cluster']=major_clusters
C:\Users\nguye\AppData\Local\Temp/ipykernel_13508/2790559756.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_cluster']=jobs_clusters
C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

changing dates and text format 

completing position end dates...
added 64030 values 

completing education end dates...
added 5715 values 

completing education start dates...
average length:  3.608725981355846 yrs
added 879 values 

completing position end dates...
average length:  3.962133030775858 yrs
added 24 values 

completing education remaining dates...
average start date:  2003.2742277481523 yrs
average end date:  2006.882953729532 yrs
added 18144 values 

completing position remaining dates...
average start date:  2010.1799402438633 yrs
average end date:  2014.1420732741742 yrs
added 6696 values 



C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\nguye\anaconda3\envs\envTF\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [28]:
X_test=np.asarray(test_data[feature_cols].values.tolist())
# matrix dim for emsemble method
X_test=np.reshape(X_test, (len(X_test), X_test.shape[1]*X_test.shape[2]))
y_pred = pipe.predict(X_test)

In [29]:
len(y_pred)

66069

In [30]:
test_data['label'] = y_pred
test_data['true_or_predicted']=False

train_data['true_or_predicted']=True

df_all=pd.concat([train_data, test_data], axis=0)

now = datetime.now()
today=now.year+now.month/13

df_all['age']=(today-df_all['label']).astype(int)
df_all=df_all[['age','true_or_predicted']]
df_all.reset_index(inplace=True)

In [31]:
tamere=df_all[~df_all['true_or_predicted']]
tamere.reset_index(drop=True)
tamere

,user_id,age,true_or_predicted
23751,++5SW5MI5/h8X1hMA3QnmQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,83,False
23752,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,58,False
23753,++6+hv3i5RAVsrWO8q5JEQ5+2cvffV/mNepQVJd0smgtpB...,27,False
23754,++7kB6m0hI1TgAPmyY1X6A5+2cvffV/mNepQVJd0smgtpB...,40,False
23755,++9DtAOTiRRvECoMIpKbmg4ZM3TcQvn1bQ/jHgHWG0kf/b...,34,False
...,...,...,...
89815,zzZdW3VGODRxRl2025ZR2w5+2cvffV/mNepQVJd0smgtpB...,34,False
89816,zzrNxfUzwZXNkSs15haLyA4ZM3TcQvn1bQ/jHgHWG0kf/b...,56,False
89817,zzrbQXjc2yHwbWjtQ9F3mg5+2cvffV/mNepQVJd0smgtpB...,32,False
89818,zzuZVPanBvW09lNk1C3h+Q5+2cvffV/mNepQVJd0smgtpB...,31,False


In [32]:
for i in range (23751,23751+5):
    user=tamere.loc[i,'user_id']
    get_user_history(user)
    print('=========')

      major   startdate     enddate
99508    BS  1949-01-01  1953-01-01
                                jobtitle   startdate enddate
41525  former_owner_presently_consultant  1953-01-01     NaN
                                jobtitle  seniority
90141  former_owner_presently_consultant   7.064817
                   major   startdate     enddate
92083  BS in Electronics  1973-01-01  1978-01-01
92505                NaN  1984-01-01  1987-01-01
                                                 jobtitle   startdate  \
9781                    owner_|_computer_network_security  1993-06-01   
106525  design_engineer_|_mechanical_industrial_engine...  1984-10-01   

           enddate  
9781           NaN  
106525  1989-05-01  
                                                 jobtitle  seniority
71129   design_engineer_|_mechanical_industrial_engine...   3.331507
222292                  owner_|_computer_network_security   7.334247
                      major   startdate     enddate
133238  Maste

## RNN and perception with Tensorflow

In [ ]:
# Import modules
from __future__ import print_function
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import numpy as np
from numpy.random import shuffle
import time
import matplotlib.pyplot as plt

# Plot configurations
%matplotlib inline
%load_ext autoreload
%autoreload 2
print(tf.__version__)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
#keeping the same split for consistency

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)

X_train=np.reshape(X_train, (len(X_train),original_shape[1],original_shape[2]))
X_train=np.swapaxes(X_train,1,2)
X_val=scaler.transform(X_val) # no fit
X_val=np.reshape(X_val, (len(X_val),original_shape[1],original_shape[2]))
X_val=np.swapaxes(X_val,1,2)
X_val.shape



In [ ]:
label_scaler=StandardScaler()
y_train=label_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
y_val=label_scaler.transform(y_val.reshape(-1, 1)).reshape(-1)

In [ ]:
X_train[0]

In [ ]:
from tensorflow.keras import layers
model = tf.keras.Sequential() 
model.add(layers.Input((original_shape[2],original_shape[1])))
model.add(layers.LSTM(1024, return_sequences=True))
model.add(layers.LSTM(256, input_shape=X.shape, return_sequences=False))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(1))

print(model.summary() )
model.compile(loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0003),
              metrics=['accuracy']) 
history_LSTM = model.fit(X_train, y_train, validation_data=(X_val, y_val),batch_size=128, epochs=100)

In [ ]:
y_pred2 = model.predict(X_val)
y_pred2 =label_scaler.inverse_transform(y_pred2.reshape(-1,1)).reshape(-1)
y_true =label_scaler.inverse_transform(y_val.reshape(-1,1)).reshape(-1)
accs=[abs(y_pred2[i]-y_true[i]) for i in range(len(y_true))]
sum(accs)/len(accs)
print('RMSE',mean_squared_error(y_true, y_pred,squared=False))

In [ ]:
X_train=np.reshape(X_train, (len(X_train),original_shape[1]*original_shape[2]))
X_val=np.reshape(X_val, (len(X_val),original_shape[1]*original_shape[2]))
X_train.shape

In [ ]:
y_val

In [ ]:
from tensorflow.keras import layers
model_perceptron = tf.keras.Sequential() 
model_perceptron.add(layers.Input((original_shape[1]*original_shape[2])))
model_perceptron.add(layers.Dense(30,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(0.001)))
model_perceptron.add(layers.Dense(20,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(0.001)))
model_perceptron.add(layers.Dense(10,activation='relu'))
model_perceptron.add(layers.Dense(5,activation='relu'))
model_perceptron.add(layers.Dense(5,activation='relu'))
model_perceptron.add(layers.Dense(10,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(0.001)))
model_perceptron.add(layers.Dense(1))

print(model_perceptron.summary() )
model_perceptron.compile(loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001),
              metrics=['accuracy']) 
history_perceptron = model_perceptron.fit(X_train, y_train, validation_data=(X_val, y_val),batch_size=128, epochs=100)

In [ ]:
y_val =label_scaler.inverse_transform(y_val.reshape(-1,1)).reshape(-1)

In [ ]:
y_pred3 = model_perceptron.predict(X_val)
print(y_pred3.shape)
y_pred3 =label_scaler.inverse_transform(y_pred3.reshape(-1,1)).reshape(-1)

accs=[abs(y_pred3[i]-y_val[i]) for i in range(len(y_val))]
print('avg abs err:',sum(accs)/len(accs))
print('RMSE',mean_squared_error(y_val, y_pred3,squared=False))

# Prediction on all the data set

# Explaination end

In [ ]:
majors_pipe=None
jobs_pipe=None

In [ ]:
# kmean test, bachelor to be removed
# data leakage but intuitively will improve the final accuracy

texts=df_education_train['major'].tolist()
len(texts)

In [ ]:
processed_text=[]
for t in texts:
    if (not pd.isnull(t)) and (not 'bachelor' in t.lower() or random.random()<0.33): # removing one third of bechelors to match the rest of the data set
        processed_text.append(t)
len(processed_text)

In [ ]:
processed_text[:20]

In [ ]:
number_of_clusters = 10
majors_pipe = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 4))), ('majors_model', KMeans(n_clusters=number_of_clusters, init='k-means++', max_iter=50, n_init=2))])
majors_pipe.fit(processed_text)



order_centroids = majors_pipe['majors_model'].cluster_centers_.argsort()[:, ::-1]
terms = majors_pipe['vectorizer'].get_feature_names_out()

for i in range(0,number_of_clusters):
    print("class %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print('-------')

In [ ]:
def add_cluster_edu(df):
    majors=df['major'].astype(str).tolist()
    major_clusters=majors_pipe.predict(majors)
    df['major_cluster']=major_clusters
add_cluster_edu(df_education_train)

In [ ]:
add_cluster_edu(df_education_val)
add_cluster_edu(df_education_test)

In [ ]:
df_education_test.head(50)

In [ ]:
df_education_train[df_education_train['major_cluster']==3].head()

In [ ]:


#Usefull functions


def to_lower_case(df,col):
    df[col] =  df[col].str.lower()
    return df 
def to_datetime(df,col):
    df[col] =  pd.to_datetime(df[col], format='%Y-%m-%d')
    df[col] =  df[col].dt.year+ df[col].dt.month/13
    return df

print('changing dates and text format \n')

df_education = df_education.pipe(to_lower_case, col='major').pipe(to_datetime,col='startdate').pipe(to_datetime,col='enddate')
df_positions = df_positions.pipe(to_lower_case, col='jobtitle').pipe(to_datetime,col='startdate').pipe(to_datetime,col='enddate')
df_seniority = df_seniority.pipe(to_lower_case, col='jobtitle')

In [ ]:
#filling missing values

def fill_end_dates(df):
    now = datetime.now()
    today=now.year+now.month/13
    #print('today',today)
    before=df['enddate'].isna().sum()
    df['enddate']=df.apply(lambda row: today if pd.isnull(row['enddate']) and (not pd.isnull(row['startdate'])) else row['enddate'],axis=1)
    print('added',before-df['enddate'].isna().sum(),'values \n')
    return df

print('completing position end dates...')

df_positions=fill_end_dates(df_positions)
print('completing education end dates...')
df_education=fill_end_dates(df_education)


In [ ]:
def fill_startdate(df):
    df_nona=df[['startdate','enddate']].dropna(how='all',inplace=False)
    df_nona['length']=df_nona['enddate']-df_nona['startdate']
    avg_length=df_nona["length"].mean()
    print('average length: ',avg_length,'yrs')
    before=df['startdate'].isna().sum()
    df['startdate'].fillna(df['enddate']-avg_length, inplace=True)
    print('added',before-df['startdate'].isna().sum(),'values \n')
    return df
print('completing education start dates...')
df_education=fill_startdate(df_education)
print('completing position end dates...')
df_positions=fill_startdate(df_positions)

In [ ]:
def fill_dates_missingall(df):
    df_nona=df[['startdate','enddate']].dropna(how='all',inplace=False)
    #df_nona['length']=df_nona['enddate']-df_nona['startdate']
    avg_start=df_nona["startdate"].mean()
    avg_end=df_nona["enddate"].mean()
    print('average start date: ',avg_start,'yrs')
    print('average end date: ',avg_end,'yrs')
    before=df['startdate'].isna().sum()
    df['startdate'].fillna(avg_start, inplace=True)
    df['enddate'].fillna(avg_end, inplace=True)
    print('added',before-df['startdate'].isna().sum(),'values \n')
    return df  
print('completing education remaining dates...')
df_education=fill_dates_missingall(df_education)
print('completing position remaining dates...')
df_positions=fill_dates_missingall(df_positions)

In [ ]:
# filling missing education and positions
df_positions['jobtitle'].fillna('positions', inplace=True)
df_seniority['jobtitle'].fillna('positions', inplace=True)
df_education['major'].fillna('education', inplace=True)

In [ ]:
df_seniority['seniority'].fillna((df_seniority['seniority'].mean()), inplace=True)

In [ ]:
# sorting by startdate for later:
df_positions.sort_values(by=['startdate'],inplace=True)
df_education.sort_values(by=['startdate'],inplace=True)

In [ ]:
print(df_education.isna().sum())
print(df_seniority.isna().sum())
df_positions.isna().sum()

In [ ]:
df=df_positions.merge(df_seniority, how='right',left_on=['user_id','jobtitle'], right_on=['user_id','jobtitle']) # only position lacks users, hence right join
df=df.groupby('user_id').agg(lambda x: list(x))
df_education_temp=df_education.groupby('user_id').agg(lambda x: list(x))
df=df.merge(df_education_temp, how='outer', left_on='user_id', right_on='user_id')
len(df)

In [ ]:
test_id ='++NWXZ8bW3I7VaXV77eiwA5+2cvffV/mNepQVJd0smgtpBr4MGMFJQ=='
print(test_id)
get_user_history(test_id)

In [ ]:
missing_count={'jobtitle': 0,'startdate_x':0,'enddate_x':0,'startdate_y':0,'startdate_y':0,'major':0,'seniority':0}
mask=[]

for rowIndex, row in df.iterrows(): 
    ismissing=False
    for i, value in row.items():
        for n in value:
            if (pd.isnull(n)):
                ismissing=True
                missing_count[i]=missing_count[i]+1
    if ismissing:
        mask.append(True)
    else:
        mask.append(False)
missing_count

In [ ]:
len([x for x in mask if x])

In [ ]:
df

In [ ]:
df['startdate_x']=df.apply(lambda row: [max(row['enddate_y']) if pd.isnull(date) else date for date in row['startdate_x'] ] ,axis=1)
now = datetime.now()
today=now.year+now.month/13
df['enddate_x']=df['enddate_x'].apply(lambda dates: [today if pd.isnull(date) else date for date in dates])